In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from numpy import sin, cos, sqrt, array, linspace
def mean_steady2b(w_m, w_L, Delta_aL, w_cLlist, g_cm, g_am, g_ac, n_th, kappa, gamma, F_L):
    N = 5    
    M = 10 
    a = tensor(destroy(N), qeye(M), qeye(2))
    b = tensor(qeye(N), destroy(M), qeye(2))
    sigma_z = tensor(qeye(N), qeye(M), sigmaz())
    sigma_p = tensor(qeye(N), qeye(M), sigmap())  
    sigma_m = tensor(qeye(N), qeye(M), sigmam())    
    num_a = a.dag() * a 
    num_b = b.dag() * b  
    c_ops = []
    gamma_a = gamma
    rate = kappa                            
    if rate > 0.0 : 
        c_ops.append(a * sqrt(rate))
    rate = n_th * gamma                      
    if rate > 0.0: 
        c_ops.append(b.dag() * sqrt(rate))
    rate = (n_th+1) * gamma
    if rate > 0.0 : 
        c_ops.append(b * sqrt(rate))
    rate = gamma_a
    if rate > 0.0 : 
        c_ops.append(sigma_m * sqrt(rate))
    idx = 0 
    n   = np.zeros(shape(w_cLlist)) 
    for w_c in w_cLlist:          
        H_Cav  = -(w_L - w_c) * num_a             
        H_Atm  =  0.5 * (Delta_aL) *sigma_z            
        H_Mech =  w_m * num_b                          
        H_Int  = 1j * g_ac * (sigma_p * a - sigma_m * a.dag())- g_cm * num_a *(b.dag() + b)- g_am * sigma_z * (b.dag() + b) 
        H_drive   =  1j* F_L *(a.dag()-a)   
        H =  H_Cav + H_Atm + H_Mech + H_Int + H_drive #+ H_0
        solvers = ['direct','power','eigen','iterative-gmres','iterative-bicgstab']  
        use_rcm = False
        rho = steadystate(H, c_ops, method=solvers[2], tol=1e-15) 
        n[idx] = expect(a.dag()*a,rho)
        idx +=1        
    H_Atm1  = 0.5 * (Delta_aL) *sigma_z
    H0      = H_Atm1 + w_m * num_b  + H_drive
    return  n, c_ops, a, H0 
w_m  = 1                       
w_a_j  = [ 15000 ]
w_L_j =  [10000 ]
g_am_j = [ 50.   ]
g_ac_j = [1.0 * 500. ]
n_th_i   = [0, 0, 1, 0 ]
gamma    = w_m/20.                                
kappa_i  = [w_m/2., 2. * w_m, w_m/2., w_m/2.]     
g_cm = 0.001 * w_m              
fig, ax= plt.subplots(1,1,figsize=(9,9))
linestyle =['-','--','--','-']
color=['black', 'black', 'r', 'b']
for j in [0]: 
    w_a  = w_a_j[j]  
    w_L  = w_L_j[j]
    Delta_aL = w_a - w_L    
    g_am_i1 = g_am_j[j] 
    g_ac = g_ac_j[j]
    g_am_i = [0, g_am_i1, g_am_i1, g_am_i1 ]
    minn_c = w_L + 60 
    maxx_c = w_L + 40 
    w_cLlist = np.linspace((minn_c )* w_m, (maxx_c ) * w_m, 301)    
    pllot1a  = np.zeros((len(w_cLlist),4))
    xplot = []    
    for i in [ 0, 1, 2, 3]:
        g_am = g_am_i[i]
        shift = g_ac**2/Delta_aL
        kappa = kappa_i[i]
        n_th            = n_th_i[i]
        F_L   = sqrt(kappa) * 0.01       
        n, c_ops, a, H0 = mean_steady2b(w_m, w_L , Delta_aL, w_cLlist, g_cm, g_am, g_ac, n_th, kappa, gamma, F_L)    
        solvers = ['direct','power','eigen','iterative-gmres','iterative-bicgstab']  
        use_rcm = False
        rho0 = steadystate(H0, c_ops, method=solvers[2], tol=1e-14)
        n_0 = expect(a.dag()*a,rho0)     
        ax.plot(w_L - w_cLlist/w_m + shift, n/n_0, linestyle =linestyle[i], color=color[i])
        ax.set_xlabel(r'$\Delta/ \omega_M $', fontsize=14)
        ax.set_ylabel(r'$\langle a^ \dagger a  \rangle /n_0 $', fontsize=14)    
        ax.set_title('Steady-state mean photon number')
        xplot = w_L - w_cLlist/w_m + shift
        pllot1a[: , i] = n/n_0     
plt.savefig("SteaSt_MeanPhoton_Hyb2b.pdf") 
xarray = xplot
black = pllot1a[: , 0]
bblack = pllot1a[: , 1]
red = pllot1a[: , 2]
blue = pllot1a[: , 3]
data1a = np.column_stack([xarray, black , bblack,red ,blue])
np.savetxt("Fig1a.txt" , data1a, fmt=['%s','%s','%s','%s','%s'])